In [5]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import cv2

def get_video_details():
    # Prompt the user to provide the video file
    print("Please provide the video file (drag and drop the file here or type the full file name):")
    video_path = input("Video File: ").strip()

    if not video_path:
        print("No file provided. Exiting.")
        return None

    # Open the selected video file
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open the video file. Please ensure the file path is correct.")
        return None

    # Extract video details if successfully opened
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    duration = frame_count / frame_rate if frame_rate else 0

    print(f"\nVideo Details:")
    print(f"- File: {video_path}")
    print(f"- Total Frames: {frame_count}")
    print(f"- Frame Rate (FPS): {frame_rate}")
    print(f"- Duration (seconds): {duration:.2f}")
    
    cap.release()
    return {
        "video_path": video_path,
        "frame_count": frame_count,
        "frame_rate": frame_rate,
        "duration": duration
    }

# Example usage
if __name__ == "__main__":
    video_details = get_video_details()
    if video_details:
        print("\nVideo processing complete!")


Please provide the video file (drag and drop the file here or type the full file name):


In [7]:
#Since its a 8.5 sec video, I choose to capture every 10th frame, after storing all the frames; I just stich them and get the panoroma shot

import os

def capture_frames(video_path, output_dir, frame_interval=10):
    """Capture every nth frame from a video and save it as an image."""
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    frame_count = 0
    saved_frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Capture every nth frame
        if frame_count % frame_interval == 0:
            # Construct the filename
            frame_filename = os.path.join(output_dir, f'frame_{saved_frame_count:04d}.jpg')
            # Save the frame as an image
            cv2.imwrite(frame_filename, frame)
            print(f'Saved: {frame_filename}')
            saved_frame_count += 1

        frame_count += 1

    cap.release()
    print("Frame capture complete.")

if __name__ == "__main__":
    video_path = '/content/video.mp4'  # Path to video file
    output_dir = 'captured_frames'          # Directory to save captured frames
    capture_frames(video_path, output_dir)

Error: Could not open video.


In [8]:
import os

# Step 1: Load Images from /content/captured_frames
def load_images(image_folder):
    images = []
    for filename in sorted(os.listdir(image_folder)):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(image_folder, filename)
            image = cv2.imread(img_path)
            if image is not None:
                images.append(image)
    return images

def stitch_images(images):
    # Create a stitcher object
    stitcher = cv2.Stitcher_create() if int(cv2.__version__.split('.')[0]) >= 4 else cv2.createStitcher()
    status, panorama = stitcher.stitch(images)

    if status == cv2.Stitcher_OK:
        return panorama
    else:
        print(f"Stitching failed with error code: {status}")
        return None

# Main Functionality
image_folder = f"/content/{output_dir}"  # Update to your directory
images = load_images(image_folder)
print(f"Loaded {len(images)} images for stitching.")

if len(images) > 1:
    panorama = stitch_images(images)
    if panorama is not None:
        output_path = "/content/stitched_panorama.jpg"
        cv2.imwrite(output_path, panorama)
        print(f"Panorama saved as '{output_path}'.")

        # Step 3: Download Panorama
        from google.colab import files
        files.download(output_path)
    else:
        print("Failed to create a panorama.")
else:
    print("Not enough images to stitch.")


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/captured_frames'